이 [링크](https://colab.research.google.com/drive/1qIFf2s2MTYc9g1Di2D07fKMq-8oBN8lc)로 들어가 파일->드라이브에 사본 저장으로 드라이브에 똑같은 주피터 노트북을 복사해주세요.

In [0]:
import torch
import torch.optim
import matplotlib.pyplot as plt
import numpy as np

In [0]:
def strange_function(x):
    # x should be torch.Tensor
    # - min(x^2, |4x|) * cos (pi x) + 2 x^2
    return - torch.min(x ** 2, torch.abs(4 * x)) * torch.cos(np.pi * x) + 2 * (x ** 2)

strange_function(torch.Tensor([1.5])) 

In [0]:
xs = np.linspace(-10, 10, 1000)

plt.plot(xs, strange_function(torch.Tensor(xs)))

### Experiment
Your job is to minimize this strange_function. As you can see, it attains its global minimum at x=0, however you should optimize this starting from x>=8. 

Choose any of gradient descents treated in lecture, then tune learning rate, momentum, and etc.

What you should do : 

1. Attain global minimum x=0.
2. Converge in x=0.
3. If possible, let epoch small as possible.

In [0]:
x = torch.tensor(8.0, requires_grad = True)

epoch = 100

optim = torch.optim.SGD([x], lr=0.01, weight_decay = 0.01, momentum = 0.01)
# optim = torch.optim.Adadelta([x], lr = 0.01)
# optim = torch.optim.Adagrad([x], lr = 0.01)
# optim = torch.optim.Adam([x], lr = 0.01)
# optim = torch.optim.RMSProp([x], lr = 0.01)

You may find that learning rate change is needed per epoch. You can do it manually, but it is better to use learning rate scheduler implemented in torch. You can see other things that I didn't write in [lr_scheduler](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)

In [0]:
scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lambda x : 0.99 * x) 
# This does lr -> 0.99 * lr -> 0.99^2 * lr -> ...

# scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size = 3, gamma = 0.1)
# This does lr -> lr * 0.1 -> lr * 0.1^2 per each step_size. i.e., 
# 0~2     3~5      6~9       10~12   ...
#  lr    lr*0.1  lr*0.1^2  lr*0.1^3  ...

# scheduler = torch.optim.lr_scheduler.MultiStepLR(optim, milestones = [5, 10, 20], gamma = 0.1)
# This does lr -> lr * 0.1 -> lr * 0.1^2 when met items in milestones. i.e., 
# 0~4     5~10     11~20      21~
# lr     lr*0.1   lr*0.1^2    lr*0.1^3

# scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.1)
# This does lr -> lr * 0.1 every epoch.


This code performs final optimizing. I recommend you to not touch this parts.

In [0]:
for i in range(epoch):
    y = strange_function(x)
    y.backward()
    grad = x.grad.item()
    optim.step()
    print("Epoch {} : x : {}, y : {}, grad : {}".format(i, x, y, grad))
    scheduler.step()